# lgo and interrupt
This notebook demonstrates you can interrupts lgo execution in Jupyter notebook.

# For-loop

In [ ]:
import (
    "fmt"
)

var sum, i int64
defer func() {
    fmt.Printf("i = %d, sum = %d, i*(i-1)/2 = %d\n", i, sum, i*(i-1)/2)
}()
for i = int64(0);; i++ {
    sum += i
}

interrupted

i = 314720394, sum = 49524463042397421, i*(i-1)/2 = 49524463042397421


# goroutine

In [ ]:
import (
    "fmt"
)

go func() {
    var i int64
    defer func() {
        fmt.Printf("i = %d (in goroutine)\n", i)
    }()
    for i = 0 ;; i++ {}
}()

i = 0 (in goroutine)
